In [55]:
import pandas as pd

import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

# all

In [56]:
q = "select count(id), payment_type from transactions where status='A' group by payment_type"
df = redshift_query_read(q)

In [57]:
df['percentage'] = df['count'] / df['count'].sum()
df['percentage'] = df['percentage'].apply(lambda x: "{:.1f}%".format(x * 100.))
df['count'] = df['count'].apply(lambda x: "{:,}".format(x))
df[['payment_type', 'percentage', 'count']]

,payment_type,percentage,count
0,VS,57.5%,"5,818,598"
1,"""  """,0.0%,"2,833"
2,None,5.8%,"583,616"
3,MC,19.6%,"1,983,785"
4,FB,0.1%,"14,624"
5,DC,3.2%,"321,568"
6,AM,7.3%,"742,123"
7,BC,0.1%,"7,763"
8,no,0.0%,"1,348"
9,ck,5.2%,"522,795"


In [58]:
df_all = df.copy()
df_all['year'] = 'all'

# past 3 years

In [59]:
q = "select count(id), payment_type, date_part('year', date) as year from transactions where status='A' and date>='2018-01-01' group by payment_type, date_part('year', date)"
df = redshift_query_read(q)

In [60]:
df['year'] = df['year'].astype(int)

In [61]:
years = df['year'].unique().tolist()
df_perced = None
for year in years:
    df_year = df[df['year']==year].copy()
    df_year['percentage'] = df_year['count'] / df_year['count'].sum()
    if df_perced is None:
        df_perced = df_year
    else:
        df_perced = df_perced.append(df_year)

In [62]:
df_perced['percentage'] = df_perced['percentage'].apply(lambda x: "{:.2f}%".format(x * 100.))
df_perced['count'] = df_perced['count'].apply(lambda x: "{:,}".format(x))

In [63]:
df_perced = df_perced.append(df_all)

In [64]:
df_perced

,count,payment_type,year,percentage
0,"94,867",AM,2018,7.34%
1,"82,051",None,2018,6.35%
4,"245,851",MC,2018,19.02%
8,"761,931",VS,2018,58.94%
11,"57,235",ck,2018,4.43%
21,"40,237",DC,2018,3.11%
22,"10,441",PP,2018,0.81%
30,125,BC,2018,0.01%
2,"51,067",ck,2021,4.09%
3,393,BC,2021,0.03%


In [67]:
df_pvtd = df_perced.pivot(index='payment_type', columns='year', values=['percentage', 'count']).reset_index()
df_pvtd

payment_type percentage                                   count           \
year                    2018    2019    2020    2021    all     2018     2019   
0             NaN      6.35%   8.11%   6.36%   7.89%   5.8%   82,051  133,390   
1            "  "        NaN     NaN     NaN     NaN   0.0%      NaN      NaN   
2              AM      7.34%   7.53%   7.42%   7.21%   7.3%   94,867  123,913   
3              BC      0.01%   0.37%   0.05%   0.03%   0.1%      125    6,135   
4              DC      3.11%   3.07%   3.31%   3.27%   3.2%   40,237   50,493   
5              FB        NaN   0.17%   0.33%   0.37%   0.1%      NaN    2,877   
6              MC     19.02%  19.00%  19.32%  19.03%  19.6%  245,851  312,642   
7              PP      0.81%   0.99%   2.24%   2.52%   1.1%   10,441   16,248   
8              VS     58.94%  56.64%  56.94%  55.58%  57.5%  761,931  931,734   
9              ck      4.43%   4.12%   4.04%   4.09%   5.2%   57,235   67,698   
10             no        NaN     NaN     NaN     NaN   0.0%      NaN      NaN   

                                     
year       2020     2021        all  
0       136,896   98,531    583,616  
1           NaN      NaN      2,833  
2       159,791   90,029    742,123  
3         1,110      393      7,763  
4        71,187   40,788    321,568  
5         7,093    4,654     14,624  
6       415,938  237,463  1,983,785  
7        48,211   31,475    111,675  
8     1,226,128  693,746  5,818,598  
9        87,050   51,067    522,795  
10          NaN      NaN      1,348

In [68]:
df_pvtd.to_csv("payment_types.year.csv", index=False)